In [1]:
from dataset import PiedmontDataset

/nfs/home/genovese/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = PiedmontDataset(root_dir='/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont')

In [ ]:
from PointDataLoader import PointDataLoader
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from dataset import PiedmontDataset

In [97]:
# CNN Feature Extractor
class SimpleCNN(nn.Module):

    def __final_Linear(self, x, size):
        return nn.Linear(size, 1)(x)



    def __init__(self, input_dim = 36, img_size=256):
        super(SimpleCNN, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_dim, 64, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.fc = self.__final_Linear

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc(x=x, size=x.size(1))

    

In [116]:
class ReduceToOnePixelNet(nn.Module):
    def __init__(self, in_channels=36, img_size=256):
        super(ReduceToOnePixelNet, self).__init__()

        layers = []

        channels = in_channels
        while(img_size > 1):  # 256 -> 128 -> ... -> 1
            # out_channels = base_channels * min(2**i, 8) 
            layers.append(nn.Conv2d(channels, channels, kernel_size=3, stride=2, padding=1))
            layers.append(nn.ReLU(inplace=True))
            img_size /= 2

        self.encoder = nn.Sequential(*layers)

    def forward(self, x):
        r = self.encoder(x)
        return r.view(1, -1)

In [48]:
a = df[0]

In [ ]:
cnn = SimpleCNN()
c = cnn.forward(a[0])

In [117]:
rrp = ReduceToOnePixelNet()
v = rrp.forward(a[0])